In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#link do folderu, w którym mamy bazę:

%cd "C:\Users\Dawid\Desktop\kurs\projekt\uber"

C:\Users\Dawid\Desktop\kurs\projekt\uber


In [3]:
occupants=pd.read_csv("./occupants.csv")
accidents=pd.read_csv("./accidents.csv")
vehicles=pd.read_csv("./vehicles.csv")

### Przeanalizowanie rekordów

#### CASEID and PSU

In [4]:
accidents['CASEID'].is_unique

False

In [5]:
accidents.groupby(["CASEID","PSU", "YEAR"])["CASEID","PSU", "YEAR"].count().head(10)

<ipython-input-5-4a2adec04204>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  accidents.groupby(["CASEID","PSU", "YEAR"])["CASEID","PSU", "YEAR"].count().head(10)


CASEID  PSU  YEAR
CASEID PSU YEAR                   
  1A   49  2009       1    1     1
       50  2005       1    1     1
  1B   9   2006       1    1     1
           2009       1    1     1
       41  2007       1    1     1
       43  2004       1    1     1
           2013       1    1     1
       79  2009       1    1     1
       81  2008       1    1     1
           2010       1    1     1

In [6]:
n_wypadkow = accidents[["CASEID","PSU","YEAR"]].drop_duplicates(subset=["CASEID","PSU","YEAR"]).shape[0]
n_wypadkow

51978

Moze byc kilka wypadkow o tym samy CASEID i w tym samym PSU, ale miały miejsce w różnych latach

### dodatkowe miary 

In [7]:
n_poszkodowanych = occupants.shape[0]

In [8]:
srednia_n_pasazerow_na_wypadek=round(n_poszkodowanych/n_wypadkow,2)
srednia_n_pasazerow_na_wypadek

2.13

Uzyskany wynik, moze oznaczać to, że z reguły wypadki toczyły się między dwoma kierowcami lub kierowcy z pasażerem, ale bez udziału drugiego samochodu. 


#### AGE

In [9]:
wypadki_per_wiek=occupants.groupby(["AGE"])['AGE'].count()/occupants.shape[0]
wypadki_per_wiek

AGE
1.0    0.025578
2.0    0.022987
3.0    0.026337
4.0    0.046073
5.0    0.170119
6.0    0.217637
7.0    0.268856
8.0    0.123901
9.0    0.072049
Name: AGE, dtype: float64

Najwiekszy udział w wypadkach mieli ludzie w wieku od 31 do 50 lat, czyli najczęstsi kierowcy, w większosci wypadków brali udział ludzie w wieku od 17 do 65 lat, stanowiło to 70% przypadków

In [10]:
print(f" W kolumnie AGE {(occupants.AGE.isna().sum()/n_poszkodowanych):.2%} przypadków to NaN ")
print(f" Przypadków NaN w AGE jest {occupants.AGE.isna().sum()}")

 W kolumnie AGE 2.65% przypadków to NaN 
 Przypadków NaN w AGE jest 2931


#### ROLE

In [11]:
wypadki_per_role=occupants.groupby(["ROLE"])['ROLE'].count()/occupants.shape[0]
wypadki_per_role

ROLE
1.0    0.660882
2.0    0.338838
Name: ROLE, dtype: float64

W ponad 33%  wypadkow biora udzial pasazerowie.

In [12]:
print(f" W kolumnie ROLE {(occupants.ROLE.isna().sum()/n_poszkodowanych):.2%} przypadków to NaN ")
print(f" Przypadków NaN w ROLE jest {occupants.ROLE.isna().sum()}")

 W kolumnie ROLE 0.03% przypadków to NaN 
 Przypadków NaN w ROLE jest 31


#### ROLE and AGE

In [13]:
wypadki_per_wiek_i_role=occupants.groupby(["AGE",'ROLE'],dropna=False)['AGE'].apply(lambda occupants: occupants.count() / n_poszkodowanych).to_frame(name = 'count').reset_index()
## moze byc tak, ze ten dopisek ,dropna=False  nie będzie działać, niby po aktualizacji powinien działać, ale może być różnice

In [14]:
wypadki_per_wiek_i_role.sort_values(by="count", ascending = False).head()

,AGE,ROLE,count
12,7.0,1.0,0.216923
9,6.0,1.0,0.160512
7,5.0,1.0,0.108236
15,8.0,1.0,0.100706
8,5.0,2.0,0.061883



Najczęściej w wypadkach biorą udział ludzie w wieku 31-50, będący kierowcami (ROLE = 1)
wsród pasażerów, najczęściej cierpią Ci w wieku 17-21


#### BAGAVAIL (Poduszka powietrzna)

In [15]:
wypadki_a_poduszka=occupants.groupby(["BAGAVAIL"])['BAGAVAIL'].count()/occupants.shape[0]
wypadki_a_poduszka

BAGAVAIL
0.0    0.185829
1.0    0.533812
Name: BAGAVAIL, dtype: float64

In [16]:
print(f" W kolumnie BAGAVAIL {(occupants.BAGAVAIL.isna().sum()/n_poszkodowanych):.2%} przypadków to NaN ")
print(f" Przypadków NaN w BAGAVAIL jest {occupants.BAGAVAIL.isna().sum()}")

 W kolumnie BAGAVAIL 28.04% przypadków to NaN 
 Przypadków NaN w BAGAVAIL jest 31052


Wiekszosc wypadkow jaka miała miejsce odbyła się u uczestników, którzy mieli poduszki potwietrzne.

### CHTYPE (fotelik)

In [17]:
wypadki_a_fotelik=occupants.groupby(["CHTYPE"])['CHTYPE'].count()/occupants.shape[0]
wypadki_a_fotelik

CHTYPE
0.0    0.760685
1.0    0.026517
Name: CHTYPE, dtype: float64

In [18]:
print(f" W kolumnie CHTYPE {(occupants.CHTYPE.isna().sum()/n_poszkodowanych):.2%} przypadków to NaN ")
print(f" Przypadków NaN w CHTYPE jest {occupants.CHTYPE.isna().sum()}")

 W kolumnie CHTYPE 21.28% przypadków to NaN 
 Przypadków NaN w CHTYPE jest 23569


### CHTYPE vs AGE

In [19]:
fotelik_a_niemowleta=occupants.loc[occupants['AGE'] == 1,"CHTYPE"].value_counts()
fotelik_a_niemowleta


1.0    1900
0.0     324
Name: CHTYPE, dtype: int64

In [20]:
print(f" Porównując potencjalnych użytkowników fotelika, tj. grupę wiekową 1 (od 0 do 3 lat) \n {fotelik_a_niemowleta[0]/sum(fotelik_a_niemowleta):.2%} wypadków miało miejsce bez fotelika")

 Porównując potencjalnych użytkowników fotelika, tj. grupę wiekową 1 (od 0 do 3 lat) 
 14.57% wypadków miało miejsce bez fotelika
